In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Make sure that caffe is on the python path:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

caffe.set_mode_gpu()

In [2]:
# Load the original network and extract the fully connected layers' parameters.
net = caffe.Net('../vgg16/VGG_ILSVRC_16_layers_deploy.prototxt', 
                '../vgg16/VGG_ILSVRC_16_layers.caffemodel', 
                caffe.TEST)
params = ['fc6', 'fc7', 'fc8']
# fc_params = {name: (weights, biases)}
fc_params = {pr: (net.params[pr][0].data, net.params[pr][1].data) for pr in params}

for fc in params:
    print '{} weights are {} dimensional and biases are {} dimensional'.format(fc, fc_params[fc][0].shape, fc_params[fc][1].shape)

fc6 weights are (4096, 25088) dimensional and biases are (4096,) dimensional
fc7 weights are (4096, 4096) dimensional and biases are (4096,) dimensional
fc8 weights are (1000, 4096) dimensional and biases are (1000,) dimensional


In [3]:
# Load the fully convolutional network to transplant the parameters.
net_full_conv = caffe.Net('person/head/train.prototxt', 
                          caffe.TEST)
params_full_conv = ['fc6', 'fc7']
# conv_params = {name: (weights, biases)}
conv_params = {pr: (net_full_conv.params[pr][0].data, net_full_conv.params[pr][1].data) for pr in params_full_conv}

for conv in params_full_conv:
    print '{} weights are {} dimensional and biases are {} dimensional'.format(conv, conv_params[conv][0].shape, conv_params[conv][1].shape)

fc6 weights are (4096, 512, 7, 7) dimensional and biases are (4096,) dimensional
fc7 weights are (4096, 4096, 1, 1) dimensional and biases are (4096,) dimensional


In [4]:
# Transplant
for pr, pr_conv in zip(params, params_full_conv):
    conv_params[pr_conv][0].flat = fc_params[pr][0].flat  # flat unrolls the arrays
    conv_params[pr_conv][1][...] = fc_params[pr][1]

In [5]:
net_full_conv.save('person/head/vgg16fc.caffemodel')